# Decorators for Downloading
## Creating a basic decorator

In [21]:
def basic(method):
    def decorated(*args, **kwargs):
        text = kwargs['text']
        print(text)
        return method(*args, **kwargs )
    return decorated

In [24]:
@basic
def basic_function(text=None):
    print("this is the function")
    return

In [25]:
basic_function(text="this is the decorator")

this is the decorator
this is the function


## Adapting the idea to optionally download and store the data

In [31]:
import requests

In [43]:
def download(method):
    def decorated(*args, **kwargs):
        if kwargs['refresh']:
            url = kwargs['url']
            dest = kwargs['dest']
            url_text = requests.get(url).text
            with open(dest, 'w+', encoding='utf-8') as f:
                f.write(url_text)
        return method(*args, **kwargs )
    return decorated        
        

In [44]:
@download
def __retrieve_data(dest=None, url=None, refresh=False):
    try:
        with open(dest, encoding='utf-8') as f:
            return f
    except FileNotFoundError:
        return __retrieve_data(dest=dest, url=url, refresh=True)
    

In [45]:
cssc = __retrieve_data(dest='cssc.txt', url='https://www.cssc.co.uk', refresh=False)

In [46]:
cssc

<_io.TextIOWrapper name='cssc.txt' mode='r' encoding='utf-8'>

## Alternative to Decorator

In [50]:
def __download_data(dest=None, url=None):
    url_text = requests.get(url).text
    with open(dest, 'w+', encoding='utf-8') as f:
        f.write(url_text)

In [51]:
def __retrieve_data_2(dest=None, url=None, refresh=False):
    if refresh:
        __download_data(dest=dest, url=url)
    try:
        with open(dest, encoding='utf-8') as f:
            return f
    except FileNotFoundError:
        return __retrieve_data(dest=dest, url=url, refresh=True)

## Alternative Decorator Design
In this version the decorator checks for file existance, instead of downloading the data. This alternative makes moer sense as the original function can always work in isolation of the decorator. 

In [1]:
def check_exist(method):
    def inner(**kwargs):
        if not kwargs['refresh']:
            try:
                with open(kwargs['loc'], encoding='utf-8') as f:
                    return f.read()
            except FileNotFoundError:
                return method(**kwargs)
        else:
            return method(**kwargs)
    return inner

In [2]:
@check_exist
def __retrieve_data_3(url=None, loc=None, refresh=False):
    url_text = requests.get(url).text
    with open(loc, 'w+', encoding='utf-8') as f:
        f.write(url_text)
    return url_text

In [3]:
cssc = __retrieve_data_3(loc='cssc.txt', url='https://www.cssc.co.uk', refresh=False)

NameError: name 'requests' is not defined